## Vector 기반의 유사도 검색 후 RAG로 공고 요약을 해보자!

In [1]:
import json
import numpy as np
import faiss

from langchain_community.vectorstores import FAISS
from sentence_transformers import SentenceTransformer
from openai import OpenAI
from getpass import getpass
from tqdm import tqdm

from utils.get_text import get_hwp_text, get_pdf_text # pdf, hwp에서 텍스트를 추출하는 사용자 지정 함수 

#### model 구현 준비
- OpenAI 객체 생성
- 파일 텍스트 추출
- model이 참고할 기본 템플릿 작성

In [2]:
# # 개인의 API key를 입력
# MY_API_KEY = getpass("OpenAI API Key")

# #OpenAI 객체 생성
# client = OpenAI(api_key=MY_API_KEY)

- 문장을 벡터로 임베딩할 모델로드

In [3]:
# Sentence-Transformers 라이브러리에서 모델 로드
# "intfloat/multilingual-e5-large" 모델은 다양한 언어에서 문장 임베딩을 생성할 수 있는 모델
model = SentenceTransformer("intfloat/multilingual-e5-large")

#### 데이터 임베딩 진행
- 데이터 로드 및 각각의 공고문 텍스트 가져오기
- 텍스트를 Vector화

In [4]:
biz_embeddings = [] # 벡터를 저장할 리스트
biz_names = [] # 공고명을 저장할 리스트

In [5]:
with open('data/bizinfo_2.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

In [6]:
for item in tqdm(data['금융']):
    file_path = item["공고파일명"]

    # 지원사업 텍스트 추출 (PDF/HWP 처리 함수 필요)
    if file_path.endswith(".pdf"):
        full_text = get_pdf_text(file_path)
    elif file_path.endswith(".hwp"):
        full_text = get_hwp_text(file_path)
    else:
        full_text = ""

    # 벡터화 및 정규화
    vector = model.encode(full_text)
    vector = vector / np.linalg.norm(vector)  # 정규화 (선택적)

    # 'vector' 키 추가
    item["vector"] = vector.tolist()  # JSON 저장을 위해 리스트로 변환


# 변환된 데이터를 JSON 파일로 저장
with open("data/data_with_vectors_2.json", "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

print("✅ 벡터가 추가된 JSON 파일 저장 완료!")

100%|██████████| 163/163 [08:24<00:00,  3.10s/it]


✅ 벡터가 추가된 JSON 파일 저장 완료!


In [7]:
for category, items in data.items():
    for item in items:
        if "vector" in item:  # 벡터 정보가 있는 경우만 추가
            biz_embeddings.append(np.array(item["vector"]))  # 리스트 → numpy 배열 변환
            biz_names.append(item["지원사업 공고명"])

# FAISS 인덱스 생성
biz_dimension = len(biz_embeddings[0])  # 벡터 차원 수
biz_index = faiss.IndexFlatL2(biz_dimension)  # L2 거리 기반 인덱스
biz_index.add(np.array(biz_embeddings))  # 벡터 추가

# ✅ 사용자 입력을 벡터화
query = '''**사업자 유형** - 개인사업자, 법인사업자, 기존사업자
:확인_표시가_있는_투표함: **지원업종** - 제조업, 정보통신업, 건설업, 운수업, 서비스업
지역 충북
신청기간 현재 진행중인 것.
대표자생년월일 x
성별 x
매출 x
종업원수 x'''
query_vector = model.encode(query).reshape(1, -1)
query_vector = query_vector / np.linalg.norm(query_vector)  # 정규화

# ✅ FAISS를 이용한 유사도 검색
top_k = 5  # 검색할 개수
distances, indices = biz_index.search(query_vector, top_k)

# ✅ 검색 결과 출력
print("🔍 검색 결과:")
for i, idx in enumerate(indices[0]):
    print(f"{i+1}. 지원사업명: {biz_names[idx]} (유사도 점수: {distances[0][i]:.4f})")

🔍 검색 결과:
1. 지원사업명: [충북] 2025년 사회적기업 육성자금 이차보전금 지원계획 공고 (유사도 점수: 0.2583)
2. 지원사업명: 2025년 소상공인 대환대출 지원계획 공고 (유사도 점수: 0.2632)
3. 지원사업명: 2025년 중소벤처기업부 소관 소상공인 정책자금 융자계획 공고 (유사도 점수: 0.2682)
4. 지원사업명: [충남] 2025년 중소기업육성자금 지원계획 공고 (유사도 점수: 0.2716)
5. 지원사업명: [충북] 2025년 중소기업육성자금 융자(이차보전) 지원계획 공고 (유사도 점수: 0.2734)
